In [1]:
from IPython.core.interactiveshell import InteractiveShell
from IPython.display import Markdown as md
from ipynb.fs.full import functions
import pandas as pd

InteractiveShell.ast_node_interactivity = "all"

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [2]:

from datetime import datetime

In [3]:
focused_case = ['PRO-3', 'PRO-4']

In [4]:
tickers_sheet = 'tickers'
tickers = pd.read_excel('output_preprocessing.xlsx', sheet_name=tickers_sheet)

timeline = pd.read_excel('timeline.xlsx', sheet_name='timeline')

pro_or_con_column_name = 'pro_or_con'
pro_or_con_choice = ['PRO', 'CON']

output_sheet_list = []

for pro_or_con in pro_or_con_choice:
    selected_pro_or_con = timeline[timeline[pro_or_con_column_name] == pro_or_con]
    for _, case in selected_pro_or_con.iterrows():
        case_code = case['code']
        if focused_case and case_code not in focused_case:
            continue

        start_date = case['formation_start_date']
        end_date = case['formation_end_date']

        start_time = datetime.strptime(start_date, '%Y-%m-%d')
        end_time = datetime.strptime(end_date, '%Y-%m-%d')

        returns_of_market = functions.generate_returns_of_market(start_time=start_time, end_time=end_time)

        tickers_stock_prices = functions.generate_stock_prices(start_time=start_time, end_time=end_time, tickers=tickers)
        tickers_return = functions.calculate_tickers_return(tickers_stock_prices=tickers_stock_prices)
            
        observation_df = functions.init_observation_df(tickers_return)
        observation_df = functions.calculate_u(observation_df, returns_of_market)
        observation_df = functions.calculate_cu(observation_df)
        sorted_observation_df = functions.sort_by_last_formation_day(observation_df)
        output_sheet_list.append({
            'sheet_name': case_code,
            'data_frame': sorted_observation_df.copy(),
        })

        del tickers_stock_prices, tickers_return, observation_df, sorted_observation_df

            


[*********************100%***********************]  598 of 598 completed
[*********************100%***********************]  598 of 598 completed


In [5]:
with pd.ExcelWriter('output_formation.xlsx') as excel_writer:
    for i in output_sheet_list:
        df = i['data_frame']
        df.index = df.index.map(str)
        sheet_name = i['sheet_name']
        df.to_excel(excel_writer=excel_writer, sheet_name=sheet_name)